In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import warnings
from datetime import *
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
#원하는 기간의 날짜를 리스트로 반환하는 함수
def date_range(start, end):
    '''
    startdate, enddate는 %Y%m%d 포맷으로 'YYYYMMDD' 형식
    '''
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y.%m.%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates


def make_soup(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup




def last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
    '''
    max_count_per_day = 크롤링할 일별 최대 뉴스의 개수
    
    
    네이버 뉴스페이지 구성

    시작페이지=1 step=10 으로 한 페이지당 10개의 뉴스를 노출시킨다.
    최대 노출 뉴스의 수는 400page 4000개의 뉴스를 노출시킨다. (그 이상은 불가능 필요하다면 검색 조건을 더 세부적으로 해야함)
    
    만약 검색한 키워드의 뉴스 총 노출수가 300개dlsep 100page를 url에 넣으면 알아서 300page로 들어가게된다

    따라서 마지막 페이지를 알아내려면 url에 page=4000을 넣으면 마지막 페이지로 이동할 수 있고
    그 soup에서 정규표현식을 통해 마지막 페이지의 숫자를 추출한다.



    날짜별로 반복:
        해당 날짜의 마지막 페이지 추출

        날짜, 마지막페이지번호, step 을 list에 추가

    return list
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #시작일 마감일 사이의 날짜들을 담은 리스트를 dates에 넣고 일 단위로 크롤링
    dates = date_range(startdate,enddate)
    
    date_lastpage_step = []

    for date in dates:  
        if max_count_per_day == -1:  
            url_ = url.format(search_keyword = search_keyword,
                            date = date,
                            date_ = date.replace('.',''),
                            page=4000)    # 네이버 뉴스페이지의 최대 노출 기사페이지 4000
            
            soup = make_soup(url_).select('a.btn') 
            
            try:    #검색 결과가 없는 경우 에러발생, 예외처리
                botton_count = int(re.findall('[0-9]+',str(soup[-1]).split(' ')[-1])[0]) * 10 - 9
                    
            except:
                print(date,'해당 일에 검색 결과가 존재하지 않습니다.')

        else:
            botton_count = max_count_per_day

        #페이지가 2이상 존재하지 않는 경우 step을 1로 설정
        step = 10 if botton_count > 10 else 1
        date_lastpage_step.append((date,botton_count,step))
    
    return date_lastpage_step



#start enddate default로 today()수정예정
def naver_news_url_crawling(search_keyword,max_count_per_day=-1,startdate='20220101',enddate='20220101'):
    '''

    max_count_per_day 하루 당 크롤링 할 최대 뉴스의 수 dafult시 (400,총 뉴스의 수) 중 작은값
    
    네이버 뉴스 웹은 page를 넘길 때 마다  (1, 11, 21, 31)순서로 바뀌고
    최대 page=4000 까지 지원한다. 페이지가 400이하일 때 page=4000을 넣으면 마지막 페이지가 나온다.
    매일 page=4000 url을 크롤링해서 마지막 페이지의 페이지버튼의 숫자를 통해 마지막 페이지를 크롤링하고
    for i in range(1, 총페이지수*10 -9, 10)으로 각 페이지마다 제목과 url 크롤링해서
    데이터프레임에 담아서 리턴 할 계획
    '''
    
    #검색하는 키워드의 띄어쓰기를 url상에 +으로 연결
    search_keyword = search_keyword.replace(' ','+')
    
    #네이버 뉴스 탭 url    
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #빈 데이터프레임을 만들고 크롤링한 데이터를 1일 단위로 concat해서 반환 할 계획
    result = []
    
    

                                #last_botton_number 파라미터는 클래스 선언하면서 추후에 수정
                                #max_count_per_day -1일 경우 최대치로 크롤링 해야하는데 수정해야함
    for date, botton_count, step in last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
        for page in range(1,botton_count+1,step): 
            url_ = url.format(search_keyword = search_keyword,
                              date = date,
                              date_= date.replace('.',''),
                              page = page)

            soup = make_soup(url_).select('a.news_tit')

            #한 페이지에 최대 노출 기사수 10개 1일 단위로 크롤링을 하니
            a_page_ulr = [html['href'] for html in soup[:min(10,max_count_per_day)]]
            a_page_title = [html['title'] for html in soup[:min(10,max_count_per_day)]]
            a_page_date = [date] * len(a_page_title)

            result.append(pd.DataFrame({'date':a_page_date, 'title':a_page_title, 'url':a_page_ulr}))
    
    result = pd.concat(result,ignore_index=True).drop_duplicates()

    # url에서 신문사의 정보를 유추할 수 있는 부분을 추출
    result['news_agency'] = result['url'].apply(lambda x : x.split('/')[2]).apply(lambda x : x.replace('www.',''))
    
    return result




def main_text_crawling():
    
    
    '''
    각 신문사의 구조가 달라서 본문을 크롤링하는데에 제한이된다.
    발행수 상위 10개의 신문사의 데이터만 사용해서 본문을 크롤링하겠다.
    '''
    return None

In [5]:
df = naver_news_url_crawling(search_keyword='교통',startdate='20220401',enddate='20220408')
df

,date,title,url,news_agency
0,2022.04.01,"서울시, 양화교 접속램프 교통통제…내년 2월까지",http://www.newsis.com/view/?id=NISX20220331_00...,newsis.com
1,2022.04.01,"교통안전공단, 변환빔 전조등시험기 특별점검",http://news.mt.co.kr/mtview.php?no=20220401112...,news.mt.co.kr
2,2022.04.01,장애인 이동권 논란에… 서울교통공사 “2024년까지 ‘1역사 1동선’ 확보”,http://www.segye.com/content/html/2022/04/01/2...,segye.com
3,2022.04.01,"정부 대표단, 싱가포르·말레이 찾아 교통인프라 협력 외교전",http://yna.kr/AKR20220331163900003?did=1195m,yna.kr
4,2022.04.01,오늘부터 접종 해외입국자 대중교통 이용 가능,https://www.sedaily.com/NewsView/264IVVWAPR,sedaily.com
...,...,...,...,...
14392,2022.04.08,[오늘의 주요 사회 일정] 대전·세종·충남(4월 8일),http://www.econonews.co.kr/news/articleView.ht...,econonews.co.kr
14393,2022.04.08,포스코건설 ESG 경영 “‘기업시민’ 이념 담아 업계 선도할 것”,http://www.smartfn.co.kr/view.php?ud=202204081...,smartfn.co.kr
14394,2022.04.08,한국건설(주) 하이엔드 아파트 브랜드 아델리움57 전속모델로 배우 고소영 선정,http://www.ikld.kr/news/articleView.html?idxno...,ikld.kr
14395,2022.04.08,"안동학교지원센터, 안동녹색어머니연합회 정기총회 개최 상시적지원",https://www.onews.tv/news/articleView.html?idx...,onews.tv


In [6]:
df['news_agency'] = df['news_agency'].apply(lambda x : 'yna.co.kr' if x == 'yna.kr' else x)

In [20]:
top10 = df['news_agency'].value_counts(normalize=True)[:30] * 100
top10

news1.kr               2.936284
newsis.com             2.741014
yna.co.kr              2.292616
fnnews.com             1.648948
news.kbs.co.kr         1.395820
news.heraldcorp.com    1.330730
newspim.com            1.287336
gukjenews.com          1.287336
view.asiae.co.kr       1.280104
shinailbo.co.kr        1.128227
sedaily.com            1.128227
nocutnews.co.kr        1.120995
news.mt.co.kr          1.034208
breaknews.com          1.026976
edaily.co.kr           1.012512
asiatoday.co.kr        0.990815
ajunews.com            0.983583
ytn.co.kr              0.961886
segye.com              0.954654
dnews.co.kr            0.853403
news.mk.co.kr          0.781080
enewstoday.co.kr       0.773848
viva100.com            0.773848
hankyung.com           0.759384
ikld.kr                0.723223
wikitree.co.kr         0.708758
news.kmib.co.kr        0.658133
moneys.mt.co.kr        0.650900
kukinews.com           0.643668
seoul.co.kr            0.643668
Name: news_agency, dtype: float64

In [29]:
top3 = top10.index[1:11]
top3

Index(['newsis.com', 'yna.co.kr', 'fnnews.com', 'news.kbs.co.kr',
       'news.heraldcorp.com', 'newspim.com', 'gukjenews.com',
       'view.asiae.co.kr', 'shinailbo.co.kr', 'sedaily.com'],
      dtype='object')

In [30]:
df1 = df[df['news_agency'].isin(top3)].reset_index(drop=True)
df1

,date,title,url,news_agency
0,2022.04.01,"서울시, 양화교 접속램프 교통통제…내년 2월까지",http://www.newsis.com/view/?id=NISX20220331_00...,newsis.com
1,2022.04.01,"정부 대표단, 싱가포르·말레이 찾아 교통인프라 협력 외교전",http://yna.kr/AKR20220331163900003?did=1195m,yna.co.kr
2,2022.04.01,오늘부터 접종 해외입국자 대중교통 이용 가능,https://www.sedaily.com/NewsView/264IVVWAPR,sedaily.com
3,2022.04.01,"""치솟는 기름값이 무서워요""…카풀·대중교통에 '원정 주유'까지",http://yna.kr/AKR20220401100700061?did=1195m,yna.co.kr
4,2022.04.01,"경상남도, 교통약자 이동서비스 확대",https://news.kbs.co.kr/news/view.do?ncd=542970...,news.kbs.co.kr
...,...,...,...,...
2141,2022.04.08,"[기자수첩] 서울시의회, 오세훈 아닌, '예산' 심사해야",http://www.newspim.com/news/view/20220407000832,newspim.com
2142,2022.04.08,여수 웅천~소호 해상교량 개통 임박…수혜 누리는 ‘웅천 트리마제 벨마레몰’,http://www.fnnews.com/news/202204080934127237,fnnews.com
2143,2022.04.08,"[최강시사] 조정식 “한동훈 무혐의 처분은 검찰식 내로남불, 유시민 징역 구형은 전...",https://news.kbs.co.kr/news/view.do?ncd=543538...,news.kbs.co.kr
2144,2022.04.08,공급 가뭄 지역에 단비... 신규 오피스텔 ‘안양 한양수자인 리버뷰’ 분양 소식에 ...,http://www.fnnews.com/news/202204080930138361,fnnews.com


In [95]:
#	newspim.com
url = 'http://www.newspim.com/news/view/20220401000711'
soup = make_soup(url)
text = soup.select('#news-contents')
result = [i[3:-4] for i in str(text).split('\n') if i[:3] == '<p>']
result

#news.heraldcorp.com
url = 'http://news.heraldcorp.com/view.php?ud=20220401000017'
soup = make_soup(url)
text = soup.select('#articleText')
result = [i.replace('</p>','') for i in str(text).split('<p>')[1:]]
result

#kbs
url = 'https://news.kbs.co.kr/news/view.do?ncd=5435528&ref=A'
soup = make_soup(url)
text = soup.select('#cont_newstext')
result = [i for i in str(text).split('<br/>') if i != ''][1:]
result

#fnnews
url = 'http://www.fnnews.com/news/202204011447482503'
soup = make_soup(url)
text = soup.select('#article_content')
result = list(filter(None,[i[5:] for i in str(text).split('\n') if i[:5] == '<br/>']))
result

#yna
url = 'https://www.yna.co.kr/view/AKR20220501015600003?input=1195m'
soup = make_soup(url)
text = soup.select('#articleWrap > div.content01.scroll-article-zone01 > div > div > article')
result = [i[4:-4]  for i in str(text).split('\n') if i[:3]=='<p>']
result


#gukjenews.com',
       'view.asiae.co.kr', 'shinailbo.co.kr', 'sedaily.com

['[헤럴드경제=신동윤 기자] 서방권에서 가장 먼저 신종 코로나바이러스 감염증(코로나19) 타격을 받은 이탈리아가 2년여 만에 본격적인 일상 회복의 첫발을 내디딘다.',
 '이탈리아는 31일(현지시간)부로 보건비상사태를 해제했다. 전 세계적으로 코로나19가 퍼지기 시작한 2020년 1월 처음 도입된 이후 약 2년 2개월 만이다.',
 '보건비상사태 해제는 코로나19 비상 체제를 마무리하고 평시로 돌아가는 신호탄으로 해석된다.',
 '그동안 코로나19 대책을 자문해온 정부 과학기술위원회와 대응 실행 기구인 코로나19 비상대책위원회도 이날부로 해산했다. 향후 모든 코로나19 관련 대책 수립·대응은 보건부로 일원화된다.',
 '보건비상사태 해제에 맞춰 4월 1일부터 방역 규제 조처도 단계적으로 풀린다.',
 '음식점·바의 야외 테이블과 박물관·미술관·우체국·은행 등의 공공시설 등에 대해선 그린 패스(방역패스)가 더는 필요치 않다.',
 '시내·시외·장거리 대중 교통수단 역시 그린 패스 대상에서 제외된다.',
 '또한 코로나19 확진자와 접촉한 사람도 백신 접종 여부와 관계없이 격리 의무에서 벗어난다. 열흘간 자가 관찰을 하고, 외출 시 우리나라의 KF94에 해당하는 FFP2 마스크 착용하는 것으로 충분하다.',
 '50세 이상 백신 미접종자 역시 코로나19 음성 확인증만 제출하면 일터에 출근할 수 있다.',
 '그동안에는 ‘슈퍼 그린패스’ 적용에 따라 백신을 맞지 않으면 출근 자체가 불가능했을뿐더러 무단결근 처리돼 그에 해당하는 급여도 받지 못했다.',
 '일선 학교의 경우 코로나19 확진자 외에 전원 대면 수업을 원칙으로 하며, 축구경기장·콘서트에는 관중·관객 100% 수용이 가능해진다.',
 '다만, 실내 음식점·바에 대한 그린패스 적용과 실내 공공장소에서의 FFP2 마스크 착용 의무는 4월 말까지 유지된다.',
 '2020년 2월 코로나19 지역 감염이 처음 확인된 이탈리아는 31일 현재 누적 확진자 1464만2354명, 총 사망자 15만9383명을 각각 기록